In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import networkx as nx
import os 
import sys
#from evaluation.mol_structure import draw_one_mol, draw_graphs
#from evaluate import evaluate
#import utils.graph_generators as gen

current = os.getcwd()
parent = os.path.dirname(current)
sys.path.append(parent)
import rdkit
import numpy as np
import json
from moses.metrics.utils  import logP, QED, SA
from rdkit import Chem
import pandas as pd
from rdkit import RDLogger 
from rdkit import Chem
from rdkit.Chem.rdchem import ChiralType, BondType, BondDir
import enum
from rdkit.Chem.rdmolops import AddHs
import networkx as nx
import matplotlib
from networkx.drawing.nx_agraph import graphviz_layout,pygraphviz_layout
from pylab import rcParams
import matplotlib as mpl
from matplotlib import pyplot as plt
import random
import numpy as np
from colour import Color
import os 
import sys

In [2]:
def rdkmol_to_nx(mol):
    #  rdkit-mol object to nx.graph
    graph = nx.Graph()
    for atom in mol.GetAtoms():
        graph.add_node(atom.GetIdx(), label=int(atom.GetAtomicNum()),    label_name=atom.GetSymbol())
    for bond in mol.GetBonds():
        graph.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(),label=(int(bond.GetBondTypeAsDouble())),
                       edge_label=str(bond.GetBondType()) )
    return graph

def list_of_smiles_to_nx_graphs(smiles):
    list_of_nx_graphs=[]
    for i,smile in enumerate(smiles):
        mol = Chem.MolFromSmiles(smile)
        if mol:
           list_of_nx_graphs.append(rdkmol_to_nx(mol))
        else:
           # print('Check smile entry no', i+1)
            list_of_nx_graphs.append(rdkmol_to_nx(Chem.MolFromSmiles('C')))
         
    return list_of_nx_graphs


In [4]:
current_smiles_datasets=['ames', 'bbb_martins', 'cyp1a2_veith', 'cyp2c19_veith','herg_karim','lipophilicity_astrazeneca']
train1_ratios=['25','33','50','66','75','90','100']
dataset_name='ames'
train1_ratio='25'

In [8]:
cd ../

c:\Users\cs843\Documents\PhD\2023\Proposal\datasets


In [9]:
split_names=[ 'train1_pos_smiles','train1_neg_smiles','test_smiles','test_targets' ]
splits={}
train1_ratios=['100']
for dataset_name in current_smiles_datasets:
    for train1_ratio in train1_ratios:
        for i,split in enumerate(split_names):
                exact_path='{}_{}/{}.txt'.format(dataset_name,train1_ratio,split)
                #from data.smiles.carcinogens import test_smiles
                current_list = []
                with open(exact_path) as my_file:
                    for line in my_file:
                        current_list.append(line.strip())
                splits[split]=current_list  
        train1_pos_smiles,train1_neg_smiles, test_smiles, test_targets=splits['train1_pos_smiles'], splits['train1_neg_smiles'],splits['test_smiles'],splits['test_targets']
        test_pos=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='1']
        test_neg=[x for x, y in zip(splits['test_smiles'],splits['test_targets']) if y=='0']
        data=train1_pos_smiles+test_pos
        mols =[Chem.MolFromSmiles(smiles)  for smiles in data]
        log_p =[logP(mol) for mol in mols]
        qeds= [QED(mol) for mol in mols]
        sas= [SA(mol) for mol in mols]
        d1={'smiles':data,'logP':log_p, 'qed':qeds,'SAS':sas}
        trial=pd.DataFrame(data=d1)
        trial.to_csv('{}_{}/{}_{}_train1_pos.csv'.format(dataset_name,train1_ratio,dataset_name,train1_ratio), index=True)
        data2=train1_neg_smiles+test_neg
        mols =[Chem.MolFromSmiles(smiles)  for smiles in data2]
        log_p =[logP(mol) for mol in mols]
        qeds= [QED(mol) for mol in mols]
        sas= [SA(mol) for mol in mols]
        d2={'smiles':data2,'logP':log_p, 'qed':qeds,'SAS':sas}
        trial=pd.DataFrame(data=d2)
        trial.to_csv('{}_{}/{}_{}_train1_neg.csv'.format(dataset_name,train1_ratio,dataset_name,train1_ratio), index=True)
        valid_idx_pos=list(np.arange (len(train1_pos_smiles), len(train1_pos_smiles)+ len(test_pos)))
        valid_idx_pos_=[str(i) for i in valid_idx_pos]
        valid_idx_neg=list(np.arange (len(train1_neg_smiles), len(train1_neg_smiles)+ len(test_neg)))
        valid_idx_neg_=[str(i) for i in valid_idx_neg]
        dict={"valid_idxs":valid_idx_pos_}
        with open("{}_{}/valid_idx_{}_{}_train1_pos.json".format(dataset_name,train1_ratio,dataset_name,train1_ratio), "w") as outfile:
            json.dump(dict, outfile)

        dict={"valid_idxs":valid_idx_neg_}
        with open("{}_{}/valid_idx_{}_{}_train1_neg.json".format(dataset_name,train1_ratio,dataset_name,train1_ratio), "w") as outfile:
            json.dump(dict, outfile)

# Section of no interest

In [17]:
from rdkit import Chem
from rdkit.Chem import GetFormalCharge
from collections  import defaultdict
import rdkit
smiles='O[C@@](CCN1CCCC1)(c1ccccc1)C1CCCCC1'
m = Chem.MolFromSmiles(smiles)
def composition(molecule):
    """Get the composition of an RDKit molecule:
    Atomic counts, including hydrogen atoms, and any charge.
    For example, fluoride ion (chemical formula F-, SMILES string [F-])
    returns {9: 1, 0: -1}.

    :param molecule: The molecule to analyze
    :type some_input: An RDKit molecule
    :rtype: A dictionary.
    """
    all_unique_atoms=set()
    # Check that there is a valid molecule
    if molecule:
        atoms=[]
        n_atoms=0
        n_atoms_with_h=0

        # Add hydrogen atoms--RDKit excludes them by default
        molecule_with_Hs = Chem.AddHs(molecule)
        comp = defaultdict(lambda: 0)
          # Get atom counts
        for atom in molecule.GetAtoms():
            n_atoms+=1
         

        # Get atom counts
        for atom in molecule_with_Hs.GetAtoms():
            comp[atom.GetAtomicNum()] += 1
            atoms.append(atom.GetAtomicNum())
            n_atoms_with_h+=1

        # If charged, add charge as "atomic number" 0
        charge = GetFormalCharge(molecule_with_Hs)
        if charge != 0:
            comp[0] = charge
        all_unique_atom=set(atoms)
        return comp,all_unique_atom,n_atoms,n_atoms_with_h
composition(m)


def get_unique_atoms(list_of_smiles):
    all_unique_atoms=set()
    max_n_atoms=0
    max_n_atoms_with_h=0
    for smile in list_of_smiles:
        try:
            m=Chem.MolFromSmiles(smile)
            #print(composition(m))
            _,unique_atoms,n_atoms,n_atoms_with_h=composition(m)
            all_unique_atoms.update(unique_atoms)
            max_n_atoms=max(max_n_atoms,n_atoms)
            max_n_atoms_with_h=max(max_n_atoms_with_h,n_atoms_with_h)
        except: 
            #print(smile)
            continue
    return all_unique_atoms,max_n_atoms_with_h,max_n_atoms


In [13]:
smniles='Cn1c(=O)nc2n(-c3ccc(N4CCOCC4)cc3)c3ccc([N+](=O)[O-])cc3cc-2c1=O'
m = Chem.MolFromSmiles(smiles)
composition(m)


(defaultdict(<function __main__.composition.<locals>.<lambda>()>,
             {8: 1, 6: 19, 7: 1, 1: 29}),
 {1, 6, 7, 8},
 21,
 50)

In [ ]:
ames_train1_pos=pd.read('data/smiles/ames/ames_train1_pos.txt')  
ames_train1_pos_smiles=list(ames_train1_pos['smiles'])
print(get_unique_atoms(ames_train1_pos_smiles))

In [14]:
get_unique_atoms([smiles])

({1, 6, 7, 8}, 21, 50)

In [7]:
cd ../

c:\Users\cs843\Documents\PhD\2023\Proposal\datasets


In [59]:
from rdkit import RDLogger
def disable_rdkit_logging():
    """
    Disables RDKit whiny logging.
    """
    import rdkit.rdBase as rkrb
    import rdkit.RDLogger as rkl
    logger = rkl.logger()
    logger.setLevel(rkl.ERROR)
    rkrb.DisableLog('rdApp.error')

results = pd.DataFrame(columns=['dataset', 'size','max_atoms','max_atoms_with_h', 'unique_atoms_list'])
results2=pd.DataFrame(columns=['dataset','train_size','train1_size','train1_pos','train1_neg','train2_size', 'test_size',])
train1_ratios=['25','33','50','66','75','90','100']
for train_ratio in train1_ratios: 
    for dataset_name in current_smiles_datasets:
        print(train_ratio,dataset_name)
        disable_rdkit_logging
        ames_train1_pos=pd.read_csv(f'{dataset_name}_{train_ratio}/{dataset_name}_{train_ratio}_train1_pos.csv')  
        ames_train1_neg=pd.read_csv(f'{dataset_name}_{train_ratio}/{dataset_name}_{train_ratio}_train1_neg.csv') 
        df = pd.read_fwf(f'{dataset_name}_{train_ratio}/train1_pos_smiles.txt')
        df2 = pd.read_fwf(f'{dataset_name}_{train_ratio}/train1_neg_smiles.txt')
        df3 = pd.read_fwf(f'{dataset_name}_{train_ratio}/train_smiles.txt')
        df5 = pd.read_fwf(f'{dataset_name}_{train_ratio}/test_smiles.txt')
        #print(len(df),len(df2),print(len(df)+len(df2)))
        ames_train1_pos_smiles=list(ames_train1_pos['smiles'])
        ames_train1_neg_smiles=list(ames_train1_neg['smiles'])
        all_unique_atoms,max_n_atoms_with_h,max_n_atoms=get_unique_atoms(ames_train1_pos_smiles)
        all_unique_atoms2,max_n_atoms_with_h2,max_n_atoms2=get_unique_atoms(ames_train1_neg_smiles)
        results = results.append(dict(dataset=dataset_name+f"_{train_ratio}_train1_pos", size=len(df),max_atoms=max_n_atoms, max_atoms_with_h=max_n_atoms_with_h,\
        unique_atoms_list=list(all_unique_atoms) ), ignore_index=True)
        results = results.append(dict(dataset=dataset_name+f"_{train_ratio}_train1_neg", size=len(df2),max_atoms=max_n_atoms2, max_atoms_with_h=max_n_atoms_with_h2,\
        unique_atoms_list=list(all_unique_atoms2) ), ignore_index=True)
        print(results)
        if train_ratio=='100':
             results2 = results2.append(dict(dataset=dataset_name+f"_{train_ratio}", train_size=len(df3),train1_size=len(df)+len(df2), train1_pos=len(df),train1_neg=len(df2),\
                                        train2_size=0,test_size=len(df5)), ignore_index=True)
        else:
            df4 = pd.read_fwf(f'{dataset_name}_{train_ratio}/train2_smiles.txt')
            results2 = results2.append(dict(dataset=dataset_name+f"_{train_ratio}", train_size=len(df3),train1_size=len(df)+len(df2), train1_pos=len(df),train1_neg=len(df2),\
                                        train2_size=len(df4),test_size=len(df5)), ignore_index=True)
        print(results2)



25 ames
              dataset size max_atoms max_atoms_with_h  \
0  ames_25_train1_pos  675        54               95   
1  ames_25_train1_neg  596        55              124   

                     unique_atoms_list  
0  [1, 35, 6, 7, 8, 9, 15, 16, 17, 53]  
1  [1, 35, 6, 7, 8, 9, 15, 16, 17, 53]  
   dataset train_size train1_size train1_pos train1_neg train2_size test_size
0  ames_25       5093        1271        675        596        3820      1455
25 bbb_martins
                     dataset size max_atoms max_atoms_with_h  \
0         ames_25_train1_pos  675        54               95   
1         ames_25_train1_neg  596        55              124   
2  bbb_martins_25_train1_pos  284        76              152   
3  bbb_martins_25_train1_neg   69       123              269   

                            unique_atoms_list  
0         [1, 35, 6, 7, 8, 9, 15, 16, 17, 53]  
1         [1, 35, 6, 7, 8, 9, 15, 16, 17, 53]  
2  [1, 5, 6, 7, 8, 9, 11, 15, 16, 17, 35, 53]  
3     [1, 35,

In [60]:
results.to_csv('statistics.csv')
results2.to_csv('statistics2.csv')


In [64]:
results

,dataset,size,max_atoms,max_atoms_with_h,unique_atoms_list
0,ames_25_train1_pos,675,54,95,"[1, 35, 6, 7, 8, 9, 15, 16, 17, 53]"
1,ames_25_train1_neg,596,55,124,"[1, 35, 6, 7, 8, 9, 15, 16, 17, 53]"
2,bbb_martins_25_train1_pos,284,76,152,"[1, 5, 6, 7, 8, 9, 11, 15, 16, 17, 35, 53]"
3,bbb_martins_25_train1_neg,69,123,269,"[1, 35, 6, 7, 8, 9, 11, 15, 16, 17, 53]"
4,cyp1a2_veith_25_train1_pos,1047,106,196,"[1, 6, 7, 8, 9, 11, 14, 15, 16, 17, 80, 28, 29..."
...,...,...,...,...,...
79,cyp2c19_veith_100_train1_neg,4802,114,226,"[1, 3, 5, 6, 7, 8, 9, 11, 14, 15, 16, 17, 19, ..."
80,herg_karim_100_train1_pos,4713,50,116,"[1, 35, 6, 7, 8, 9, 14, 15, 16, 17, 53]"
81,herg_karim_100_train1_neg,4697,58,135,"[1, 5, 6, 7, 8, 9, 11, 14, 15, 16, 17, 79, 34,..."
82,lipophilicity_astrazeneca_100_train1_pos,2445,72,128,"[1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 34, 35, 53]"


In [62]:
results2

,dataset,train_size,train1_size,train1_pos,train1_neg,train2_size,test_size
0,ames_25,5093,1271,675,596,3820,1455
1,bbb_martins_25,1420,353,284,69,1065,405
2,cyp1a2_veith_25,8804,2199,1047,1152,6603,2515
3,cyp2c19_veith_25,8865,2214,985,1229,6649,2532
4,herg_karim_25,9411,2351,1151,1200,7058,2688
5,lipophilicity_astrazeneca_25,2939,733,628,105,2204,839
6,ames_33,5093,1696,900,796,3395,1455
7,bbb_martins_33,1420,471,360,111,947,405
8,cyp1a2_veith_33,8804,2933,1357,1576,5869,2515
9,cyp2c19_veith_33,8865,2953,1331,1622,5910,2532


In [67]:
x=[i for i in range(19, 19+84)]
print(x)

[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [21]:
 for smile in new_smiles:
        m=Chem.MolFromSmiles(smile)             
        params = Chem.RemoveHsParameters()
        params.removeDegreeZero = True
        mol_nohs = Chem.RemoveHs(m, params)
        

# Section